<a href="https://colab.research.google.com/github/mahb97/joyce-dubliners-similes-analysis/blob/main/notebooks/02_gutenberg_extraction_and_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Gutenberg Dubliners - Simile Extraction and Processing

This notebook extracts similes from the Project Gutenberg version of Dubliners and performs:
- Comprehensive simile extraction using all identified patterns
- Lemmatization and POS tagging
- Topic modeling
- Sentiment analysis
- Data structuring for analysis

In [ ]:
# Setup and imports
import spacy
import pandas as pd
import requests
import re
import numpy as np
from textblob import TextBlob

print("Starting setup...")

In [ ]:
# Install required packages
!pip install spacy textblob scikit-learn
!python -m spacy download en_core_web_lg
!python -m textblob.download_corpora

## Load Project Gutenberg Text

In [ ]:
# Text loading functions will go here
pass